## Task 1: Exploratory Data Analysis (EDA) of Financial News

This notebook performs Task 1 of the project:
- Descriptive statistics of news headlines
- Text preprocessing and cleaning
- Time series analysis of news publication
- Publisher analysis


In [2]:
import sys
from pathlib import Path

# Add project root and src/ folder to Python path
project_root = Path("..").resolve()
src_path = project_root / "src"

sys.path.append(str(project_root))
sys.path.append(str(src_path))

print("Project root:", project_root)
print("SRC path:", src_path)

Project root: D:\10Acadamy\Week 1\Task\Predicting-Price-Moves-with-News-Sentiment
SRC path: D:\10Acadamy\Week 1\Task\Predicting-Price-Moves-with-News-Sentiment\src


### Imports


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from fns_project.data.loader import load_news_csv
from fns_project.data.preprocess import preprocess_headlines
from fns_project.data.align_dates import normalize_publication_dates

from fns_project.viz.plots import plot_daily_counts
from fns_project.nlp.topic_modeling import (
    extract_keywords,
    lda_topics
)
from fns_project.analysis.stats import (
    add_headline_length,
    publisher_distribution
)

sns.set(style="whitegrid")

news_path = Path("../data/raw_sample/news.csv")

ImportError: cannot import name 'normalize_publication_dates' from 'fns_project.data.align_dates' (D:\10Acadamy\Week 1\Task\Predicting-Price-Moves-with-News-Sentiment\src\fns_project\data\align_dates.py)